# **Add your Kaggle API key in the choose file section after running the below cell which is available on your kaggle setting inorder to connect the colab.**

In [ ]:
# Upload your own Kaggle API key from https://www.kaggle.com/account
# from google.colab import files
# files.upload()  # Upload kaggle.json


Make .kaggle/ directory on this session and the change access of the directory through chmod 600 and then join the .kaggle/ dir as like this .kaggle/kaggle.json

If you want to store the dataset in some persistent storage area you can use Gdrive. So mount the drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
#drive.mount('/content/drive')

# Note Use can use (!cp source destination) command to load the dataset to the drive location

# Define the path to the dataset directory in Google Drive storing the dataset in the drive
# IMPORTANT: Users should update this path to where they have stored the HAM10000 dataset
image_path = "/content/drive/MyDrive/Colab_Notebooks/ham10000" # <--- Update this path

# Verify that the directory exists and list some files
if os.path.exists(image_path):
    print(f"Dataset directory found at: {image_path}")
    print("First 5 files in the directory:", os.listdir(image_path)[:5])
else:
    print(f"Error: Dataset directory not found at {image_path}")
    print("Please update the 'image_path' variable to the correct location of the HAM10000 dataset in your Google Drive.")

 Below it is a sample cell to check whether I can access the image file

In [ ]:
import PIL.Image as Image
import matplotlib.pyplot as plt

image_path = '/content/drive/MyDrive/Colab_Notebooks/ham10000/HAM10000_images_part_1/ISIC_0024306.jpg'
image = Image.open(image_path)

plt.imshow(image)
plt.title('Example Image')
plt.axis('off')
plt.show()

Mapping all the target values into numbers for model training

In [ ]:
label2id = {
    "akiec": 0,
    "bcc": 1,
    "bkl": 2,
    "df": 3,
    "mel": 4,
    "nv": 5,
    "vasc": 6
}

id2label = {v:k for k, v in label2id.items()}


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Define the path to the metadata CSV file in Google Drive
# IMPORTANT: Users should update this path to where they have stored the HAM10000_metadata.csv file
metadata_path = "/content/drive/MyDrive/Colab_Notebooks/ham10000/HAM10000_metadata.csv" # <--- Update this path

# Define the base path to the image directory in Google Drive
# Ensure this matches the path defined in the previous cell
image_base_path = "/content/drive/MyDrive/Colab_Notebooks/ham10000" # <--- Ensure this matches the path in cell 061QDw563n66

df = pd.read_csv(metadata_path)

# Construct the full image path using the base path in image_id
df['image_path'] = df['image_id'].apply(lambda x: os.path.join(image_base_path, f"HAM10000_images_part_1/{x}.jpg")) # Adjust subfolder if necessary

# Map the string labels in dx field to number labels in new 'label' field in df
df['label'] = df['dx'].map(label2id)


# Filter out rows where the image file does not exist
df = df[df['image_path'].apply(lambda x: os.path.exists(x))]

train_df , test_df = train_test_split(df, test_size=0.2, stratify = df['label'], random_state = 0)

In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification

# Define the name of the pre-trained model from Hugging Face
model_name = 'google/vit-base-patch16-224-in21k'

# Load the image processor associated with the pre-trained model
# This processor will handle image resizing, normalization, etc.
image_processor = AutoImageProcessor.from_pretrained(model_name)

# Load the pre-trained model for image classification
image_model = AutoModelForImageClassification.from_pretrained(
    model_name,
    num_labels = len(label2id),
    label2id = label2id,
    id2label = id2label
    )

In [ ]:
#building a custom dataset from dataset class

from torch.utils.data import Dataset
import torch

# Define a custom PyTorch Dataset for loading skin cancer images and labels
class SkinCancerDataset(Dataset):

  # Constructor: initializes the dataset with a DataFrame and an image processor
  def __init__(self, df, image_processor):
    self.df = df.reset_index(drop=True)
    self.image_processor = image_processor

  # Returns the total number of samples in the dataset
  def __len__(self):
    return len(self.df)

  # Returns a single sample (image and label) at the given index
  def __getitem__(self, idx):
    # Open the image file, convert to RGB format
    image = Image.open(self.df.loc[idx, 'image_path']).convert("RGB")
    # Get the corresponding label (integer id)
    label = self.df.loc[idx, 'label']

    # Preprocess the image using the image processor, return as PyTorch tensors
    inputs = self.image_processor(images=image, return_tensors="pt")
    # Remove the batch dimension added by the image processor
    inputs = {k:v.squeeze(0) for k, v in inputs.items()}
    # Add the label as a PyTorch tensor to the inputs dictionary
    inputs["labels"] = torch.tensor(label)

    # Return the dictionary containing processed image inputs and the label
    return inputs

In [ ]:
from torch.utils.data import DataLoader

# Create instances of the custom SkinCancerDataset for the training and test dataframes

new_traindf = SkinCancerDataset(train_df, image_processor)
new_testdf = SkinCancerDataset(test_df, image_processor)

# These objects will handle loading and preprocessing images during training and evaluation

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
import numpy as np # Import numpy for the compute_metrics function
from sklearn.metrics import accuracy_score # Import accuracy_score as well

# Define the compute_metrics function
# This function is used to calculate metrics during evaluation

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions."""
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Define the training arguments
# These arguments configure the training process, such as batch size, epochs, evaluation strategy, etc.

training_args = TrainingArguments(
    output_dir='./result',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir='./logs',
    report_to="wandb",
    fp16=True
)

# Initialize the Trainer
# The Trainer handles the training and evaluation loop
trainer = Trainer(
    model=image_model,
    args=training_args,
    train_dataset=new_traindf,
    eval_dataset=new_testdf,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [ ]:
trainer.train() # Finally Train the model this will take some time so enable a gpu for faster performance

In [ ]:
import shutil
import os

# Define the source and destination paths
source_dir = './result'  # The output directory specified in TrainingArguments
destination_dir = '/content/drive/MyDrive/Colab_Notebooks/ham10000/trained_model' # Replace with your desired path in Google Drive

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Copy the contents of the source directory to the destination directory
shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)

print(f"Model saved to: {destination_dir}")

In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification

# Define the path to the saved model in your Google Drive
model_path = '/content/drive/MyDrive/Colab_Notebooks/ham10000/trained_model/checkpoint-1500'

# Load the image processor
loaded_image_processor = AutoImageProcessor.from_pretrained('/content/drive/MyDrive/Colab_Notebooks/ham10000/trained_model/checkpoint-1500/config.json')

# Load the model
loaded_model = AutoModelForImageClassification.from_pretrained(model_path)

print("Model and image processor loaded successfully!")

In [ ]:
import torch.nn.functional as F
import pandas as pd
import random
from PIL import Image # Import Image

# Select a random image path from the test dataset
new_image_path = random.choice(test_df['image_path'].tolist())

# Open the selected image and convert it to RGB format
image = Image.open(new_image_path).convert("RGB")

# Find the information for this image in the original dataframe to get the actual label
image_info = df[df['image_path'] == new_image_path]

# Check if the image information was found and print the actual label
if not image_info.empty:
    actual_label_id = image_info.iloc[0]['label']
    actual_label = id2label[actual_label_id]
    print(f"Image path: {new_image_path}")
    print(f"The actual skin cancer type for the image is: {actual_label}")
else:
    actual_label = "N/A (Image not found in original data)"
    print(f"Image path: {new_image_path}")
    print(f"The actual skin cancer type for the image is: {actual_label}")

# Preprocess the image using the loaded image processor
# This prepares the image for the model input
inputs = loaded_image_processor(images=image, return_tensors="pt")

# Determine the device (GPU or CPU) to run the inference on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loaded_model.to(device) # Move the loaded model to the selected device

inputs = {k: v.to(device) for k, v in inputs.items()} # Move the input tensors to the selected device

# Perform inference using the loaded model
# torch.no_grad() is used to disable gradient calculation during inference to save memory and speed up computation
with torch.no_grad():
    outputs = loaded_model(**inputs)

# Get the logits (raw output scores) from the model's output
logits = outputs.logits

# Get the predicted class ID by finding the index of the maximum logit score
predicted_class_id = torch.argmax(logits, dim=1).item()

# Calculate the probabilities for each class using softmax
probabilities = F.softmax(logits, dim=1)[0]

# Get the probability for the predicted class
predicted_probability = probabilities[predicted_class_id].item()

# Map the predicted class ID back to its string label
predicted_label = id2label[predicted_class_id]

# Print the predicted skin cancer type and its confidence score
print(f"The predicted skin cancer type from the model is: {predicted_label}")
print(f"Confidence score (probability): {predicted_probability:.4f}")

# **Run the last cell multiple times for checking the prediction score / confidence score... Each time a new image will be picked from the validation dataset and see if  the original label from the dataset corresponds to my trained model**